In [3]:
import Pkg; Pkg.activate(".")
using Plots
using StatsBase

Activating environment at `~/teaching/network_labs/Project.toml`


In [4]:
# Create a folder called data if it doesn't exist
ispath(joinpath("data", "knn")) || mkdir(joinpath("data", "knn"))

true

In [5]:
_project_root = "https://raw.githubusercontent.com/PhDP/EcoInter/master/KNN/data/"
isfile(joinpath("data", "knn", "interactions.csv")) || download(_project_root*"mercure-interactions.csv", joinpath("data", "knn", "interactions.csv"))
isfile(joinpath("data", "knn", "traits-codes.txt")) || download(_project_root*"mercure-trait-codes.txt", joinpath("data", "knn", "traits-codes.txt"))
isfile(joinpath("data", "knn", "traits.csv")) || download(_project_root*"mercure-traits.csv", joinpath("data", "knn", "traits.csv"))

true

In [6]:
data_interactions = joinpath("data", "knn", "interactions.csv")
preys = Set{Int64}[]

0-element Array{Set{Int64},1}

In [7]:
for (i,line) in enumerate(readlines(data_interactions))
    if length(line) > 0
        this_preys = parse.(Int64, split(line, ", "))
        push!(preys, Set(this_preys))
    else
        push!(preys, Set(Int64[]))
    end
end

In [8]:
preys[1:5]

5-element Array{Set{Int64},1}:
 Set([405, 432, 391, 392])          
 Set([405, 432, 391, 392])          
 Set([403, 394, 395, 401, 404, 402])
 Set([403, 394, 395, 401, 404, 402])
 Set([403, 394, 395, 401, 404, 402])

In [9]:
tanimoto(s1::T, s2::T) where {T <: Set} = length(s1 ∩ s2)/length(s1 ∪ s2)

tanimoto (generic function with 1 method)

In [10]:
D(s1,s2) = 1.0 - tanimoto(s1,s2)

D (generic function with 1 method)

In [110]:
top1 = Bool[]
top3 = Bool[]
top5 = Bool[]
top10 = Bool[]
neighbors_to_use = 5
for prey_idx in 1:length(preys)
    if length(preys[prey_idx]) > 2
        sampled_preys = sample(collect(preys[prey_idx]), length(preys[prey_idx])-1, replace=false)
        removed_prey = first(collect(filter(p -> !(p in sampled_preys), preys[prey_idx])))
        test_preys = preys[filter(i -> i != prey_idx, 1:length(preys))]
        neighbors = [D(Set(sampled_preys), p) for p in test_preys]
        likely_preys = test_preys[partialsortperm(neighbors, 1:neighbors_to_use)]
        recos = countmap(vcat(collect.(likely_preys)...))
        recommended = zeros(Bool, top_recos)
        for re in 1:10
            if length(recos) > 0
                cmax = filter(f -> f.second == maximum(values(recos)), recos)
                recommended[re] = removed_prey in collect(keys(cmax))
                filter!(f -> f.second < maximum(values(recos)), recos)
            end
        end
        push!(top1, sum(recommended[1]))
        push!(top3, sum(recommended[1:3]))
        push!(top5, sum(recommended[1:5]))
        push!(top10, sum(recommended[1:10]))
    end
end

In [111]:
@info "Top  1 success, k=$(neighbors_to_use): $(round(sum(top1)/length(top1).*100; digits=2))%"
@info "Top  3 success, k=$(neighbors_to_use): $(round(sum(top3)/length(top3).*100; digits=2))%"
@info "Top  5 success, k=$(neighbors_to_use): $(round(sum(top5)/length(top5).*100; digits=2))%"
@info "Top 10 success, k=$(neighbors_to_use): $(round(sum(top10)/length(top10).*100; digits=2))%"

┌ Info: Top  1 success, k=5: 51.02%
└ @ Main In[111]:1
┌ Info: Top  3 success, k=5: 73.62%
└ @ Main In[111]:2
┌ Info: Top  5 success, k=5: 83.97%
└ @ Main In[111]:3
┌ Info: Top 10 success, k=5: 83.97%
└ @ Main In[111]:4
